In [1]:
"""
Filter index based on gene name.
"""
import os
import pandas as pd
_chr_indices = [
    'NC_000001.11.csv',
    'NC_000002.12.csv',
    'NC_000003.12.csv',
    'NC_000004.12.csv',
    'NC_000005.10.csv',
    'NC_000006.12.csv',
    'NC_000007.14.csv',
    'NC_000008.11.csv',
    'NC_000009.12.csv',
    'NC_000010.11.csv',
    'NC_000011.10.csv',
    'NC_000012.12.csv',
    'NC_000013.11.csv',
    'NC_000014.9.csv',
    'NC_000015.10.csv',
    'NC_000016.10.csv',
    'NC_000017.11.csv',
    'NC_000018.10.csv',
    'NC_000019.10.csv',
    'NC_000020.11.csv',
    'NC_000021.9.csv',
    'NC_000022.11.csv',
    'NC_000023.11.csv',
    'NC_000024.10.csv']
_chr_dir = ["chr{}".format(i+1) for i in range(len(_chr_indices))]

for chr, chr_number in zip(_chr_indices, _chr_dir):
    path = os.path.join('data', 'genome', 'grch38', 'csvs_strand', chr)
    df = pd.read_csv(path)
    genes = list(df['gene'].unique())
    genes = [a for a in genes if not pd.isnull(a)]
    for g in genes:
        ndf = df[df['gene'] == g]
        path = os.path.join('data', 'genome', 'grch38', 'genes', chr_number,"{}.csv".format(g))
        if not os.path.exists(os.path.dirname(path)):
            os.mkdir(os.path.dirname(path))
        ndf.to_csv(path, index=False)
        print("Success: {}                                                      ".format(path), end="\r")

C:\Users\lmanw\AppData\Local\Temp\ipykernel_21168\3840047348.py:35: DtypeWarning: Columns (7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [4]:
"""
Generate sequence from fasta based on gene indices.
"""
import os
from Bio import SeqIO
import pandas as pd
from tqdm import tqdm
from data_dir import chr_fasta_mapname

strand = '+'
chrs = ['chr{}'.format(i+1) for i in range(24)] # Folder name.

for c in chrs:
    chr_genes_indices_dir = os.path.join("data", "genome", "grch38", "genes", c)
    chr_genes_sequence_csv_dir = os.path.join("data", "genome", "sequential-labelling-positive-strand",c)
    chr_fasta = os.path.join("data", "chr", chr_fasta_mapname[c])

#chr1_genes_indices_dir = os.path.join("data", "genome", "grch38", "genes", "chr1")
#chr1_genes_sequence_csv_dir = os.path.join("data", "genome", "sequential-labelling", "chr1")
#chr1_fasta = os.path.join("data", "chr", "NC_000001.11.fasta")

    if not os.path.exists(chr_fasta):
        raise FileNotFoundError("Fasta {} not found.".format(chr_fasta))

    """
    Read whole chromosome here and return its complete sequence.
    Yes, it's long.
    """
    records = SeqIO.parse(chr_fasta, "fasta")
    chr_records = next(records)
    chr_sequence = str(chr_records.seq)
    genome_sequence = chr_sequence # Use genome sequence reader here.

    _columns = ['sequence', 'label']
    for fname in os.listdir(chr_genes_indices_dir):
        print("Working on chr {} gene {}                                    ".format(c, fname), end='\r')
        fpath = os.path.join(chr_genes_indices_dir, fname)
        if os.path.isfile(fpath):
            """
            Gene index found. Read the index and cross-reference with genome sequence.
            """
            index_df = pd.read_csv(fpath)
            gene_region = index_df[index_df['region'] == "gene"]

            if len(gene_region) > 0:
                gene_df = pd.DataFrame(columns=_columns)
                for i, g in gene_region.iterrows():
                    gene_start_index = int(g['start_index'])
                    gene_end_index = int(g['end_index'])
                    gene_sequence = genome_sequence[gene_start_index:gene_end_index + 1]
                    # print("Gene sequence: {}".format(gene_sequence))
                    # If gene sequence isn't None then this gene is available in chromosome sequence.
                    if gene_sequence != None:
                        gene_sequential_labelling = ['.' for a in gene_sequence]
                        exons = index_df[index_df['region'] == "exon"]
                        if strand != None:
                            exons = index_df[index_df['strand'] == strand]
                        for i, r in exons.iterrows():
                            start_index = int(r['start_index'])
                            end_index = int(r['end_index'])
                            if (start_index >= gene_start_index and end_index <= gene_end_index):
                                start_index = int(r['start_index']) - gene_start_index
                                end_index = int(r['end_index']) - gene_start_index
                                for i in range(start_index, end_index + 1):
                                    gene_sequential_labelling[i] = 'E'
                        gene_sequential_labelling = ''.join(gene_sequential_labelling)
                        #endfor
                        gene_df = pd.concat([gene_df, pd.DataFrame([[gene_sequence, gene_sequential_labelling]], columns=_columns)])

                target_path = os.path.join(chr_genes_sequence_csv_dir, fname)
                # data\genome\sequential-labelling\chr1
                # Only write of dataframe is not empty.
                if gene_df.shape[0] > 0:
                    if not os.path.exists(chr_genes_sequence_csv_dir):
                        os.makedirs(chr_genes_sequence_csv_dir, exist_ok=True)    
                    gene_df.to_csv(target_path, index=False)
                    

In [5]:
"""
Generate gene index in CSV ('chr', 'gene') to see if dataset split can be done by genes.
"""
import pandas as pd

chrs = ['chr{}'.format(i+1) for i in range(24)]
pos_strand = os.path.join("data", "genome", "sequential-labelling-positive-strand")
chrs = [os.path.join(pos_strand, c) for c in chrs]

genes = []
cs = []
_cols = ['chr', 'gene']
df = pd.DataFrame(columns=_cols)
for path in chrs:
    (head, tail) = os.path.split(path)
    for gindex in os.listdir(path):
        gindex = gindex.split('.')
        gindex = gindex[0]
        genes.append(gindex)
        cs.append(tail)
#endfor
df['chr'] = cs
df['gene'] = genes
tpath = os.path.join('workspace', 'sequential-labelling', 'gene_index.csv')
os.makedirs(os.path.dirname(tpath), exist_ok=True)
df.to_csv(
    tpath,
    index=False
)

In [11]:
import pandas as pd
_cols = ['chr', 'gene']
df_path = os.path.join('workspace', 'sequential-labelling', 'gene_index.csv')
_dir = os.path.join('workspace', 'sequential-labelling', 'duplicate genes')
os.makedirs(_dir, exist_ok=True)
df = pd.read_csv(df_path)
train_df = pd.DataFrame(columns=_cols)
valid_df = pd.DataFrame(columns=_cols)
test_df = pd.DataFrame(columns=_cols)
genes_unique = df['gene'].unique()
for g in genes_unique:
    filtered_df = df[df['gene'] == g]
    if filtered_df.shape[0] > 1:
        _g = "{}.csv".format(g)
        filtered_df.to_csv(os.path.join(_dir, _g), index=False)

In [ ]:
import os

gene_index_dir = os.path.join('workspace', 'sequential-labelling')
gene_index = os.path.join(gene_index_dir, 'gene_index.csv')


In [2]:
from data_preparation import gff_to_csvs, gff_to_csv
from data_dir import (annotated_grch38_gff, annotated_grch38_gff_dir, annotated_grch38_gff_csv)

print(annotated_grch38_gff)
print(annotated_grch38_gff_csv)
print(annotated_grch38_gff_dir)

./data/genome/grch38/GRCh38_latest_genomic.gff
./data/genome/grch38/grch38_gff.csv
./data/genome/grch38/csvs


In [2]:
from data_dir import chr21_fasta, chr21_index_csv, data_genome_grch38_labels_dir
from data_preparation import generate_sequence_labelling
import os

target_path = os.path.join(data_genome_grch38_labels_dir, 'chr21.csv')
print("Generate sequential labelling {} => {}: {}".format(chr21_index_csv, target_path, generate_sequence_labelling(chr21_index_csv, chr21_fasta, target_path)))


Generate sequential labelling data/genome/grch38/csvs/NC_000021.9.csv => data/genome/grch38/labels\chr21.csv: True data/genome/grch38/labels\chr21.csv


In [4]:
from data_dir import (
    chr1_index_csv, chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
)
from data_dir import (
    chr1_fasta, chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta, 
	chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta, 
	chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta,
)
chr_fastas = [
    chr1_fasta,
	#chr2_fasta, chr3_fasta, chr4_fasta, chr5_fasta, chr6_fasta, chr7_fasta, chr8_fasta, chr9_fasta, chr10_fasta,
	#chr11_fasta, chr12_fasta, chr13_fasta, chr14_fasta, chr15_fasta, chr16_fasta, chr17_fasta, chr18_fasta, chr19_fasta, chr20_fasta,
	#chr21_fasta, chr22_fasta, chr23_fasta, chr24_fasta
]
from data_dir import labseq_dir, labseq_names
from data_preparation import generate_sequence_labelling
chr_indices = [
    chr1_index_csv, 
	#chr2_index_csv, chr3_index_csv, chr4_index_csv, chr5_index_csv, chr6_index_csv, chr7_index_csv, chr8_index_csv, chr9_index_csv, chr10_index_csv,
    #chr11_index_csv, chr12_index_csv, chr13_index_csv, chr14_index_csv, chr15_index_csv, chr16_index_csv, chr17_index_csv, chr18_index_csv, chr19_index_csv, chr20_index_csv,
    #chr21_index_csv, chr22_index_csv, chr23_index_csv, chr24_index_csv
]
chr_labseq_path = [os.path.join(labseq_dir, fname) for fname in labseq_names[0:1]]
for src, fasta, target in zip(chr_indices, chr_fastas, chr_labseq_path):
    print(src, fasta, target)
    #print("Generating sequential labelling for index {}, from fasta {}, to {}: {}".format(src, fasta, target, generate_sequence_labelling(src, fasta, target, do_expand=True, expand_size=512)))

data/genome/grch38/csvs/NC_000001.11.csv data/chr/NC_000001.11.fasta data/genome/labseq\chr1.csv


In [2]:
from sequential_labelling import Label_Dictionary
import pandas as pd
from tqdm import tqdm
import os

def prepare_sequence_from_csv(src_csv, label_dictionary=Label_Dictionary):
    """
    Convert sequence into tokenized DNA sequence and label sequence.
    CSV source has columns `sequence` and `label`. 
    `sequence` contains tokenized DNA sequence and `label` contains sequence of labels.
    @param      src_csv (string): path to CSV source.
    @param      label_dictionary (dict): dictionary to convert label into number.
    @return     sequence, labels
    """
    if not os.path.exists(src_csv):
        raise FileNotFoundError(src_csv)
    if label_dictionary == None:
        raise Exception("Argument `label_dictionary` cannot be empty!")
    
    df = pd.read_csv(src_csv)
    return list(df['sequence']), list(df['label'])
    

['a', 'a', 'a', 'a', 'a', 'b', 'a', 'a', 'a', 'a']